In [1]:
import torch
import wandb
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR
from sklearn.model_selection import train_test_split
from torch.utils.data.sampler import SubsetRandomSampler


In [2]:
train_path = "/kaggle/input/inatural/inaturalist_12K/train"
test_path = "/kaggle/input/inatural/inaturalist_12K/val"
def load_data(train_path, val_split=0.2, batch_size=150, data_augmentation=False):
    transform_list = [transforms.Resize((224, 224)),transforms.ToTensor()]
    if data_augmentation:
        transform_list.insert(0, transforms.RandomHorizontalFlip())
        transform_list.insert(0, transforms.RandomRotation(10))

    transform = transforms.Compose(transform_list)

    # Load dataset
    dataset = ImageFolder(train_path, transform=transform)
    num_classes = len(dataset.classes)
    
    # Create stratified train-validation split
    train_indices, val_indices = train_test_split(list(range(len(dataset))), 
                                                   test_size=val_split, 
                                                   stratify=dataset.targets)

    # Create DataLoader for training set
    train_sampler = SubsetRandomSampler(train_indices)
    train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)

    # Create DataLoader for validation set
    val_sampler = SubsetRandomSampler(val_indices)
    val_loader = DataLoader(dataset, batch_size=batch_size, sampler=val_sampler)

    return train_loader, val_loader, num_classes

In [ ]:
tl,vl,n = load_data(train_path, val_split=0.2, batch_size=1000, data_augmentation=False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
for batch_idx, (images, labels) in enumerate(vl):
    print(f"Batch {batch_idx + 1}:")
    print("Images shape:", images.shape)
    print("Labels shape:", labels.shape)

In [3]:
!wandb login "8506a95d562352ae82e224bf1d33b8cdf34c0c0e" 

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [5]:
class CNN(nn.Module):
    def __init__(self, input_channels, num_classes, num_filters, filter_size, dense_units, activation_conv='relu', data_augmentation=False, batch_norm=False, dropout=0.0, filter_organization='same'):
        super(CNN, self).__init__()

        self.conv_layers = nn.ModuleList()
        for i in range(5):
            if filter_organization == 'same':
                current_num_filters = num_filters
            elif filter_organization == 'double':
                current_num_filters = num_filters * (2 ** i)
            elif filter_organization == 'halve':
                current_num_filters = num_filters // (2 ** i)

            self.conv_layers.append(nn.Conv2d(input_channels, current_num_filters, filter_size, padding=1))
            if batch_norm:
                self.conv_layers.append(nn.BatchNorm2d(current_num_filters))
            if activation_conv == 'relu':
                self.conv_layers.append(nn.ReLU(inplace=True))
            elif activation_conv == 'gelu':
                self.conv_layers.append(nn.GELU())
            elif activation_conv == 'silu':
                self.conv_layers.append(nn.SiLU())
            elif activation_conv == 'mish':
                self.conv_layers.append(nn.Mish())
            self.conv_layers.append(nn.Dropout2d(dropout))
            self.conv_layers.append(nn.MaxPool2d(kernel_size=2))
            input_channels = current_num_filters

        input_size = 224  # Assuming input images are 224x224
        for _ in range(5):
            #input_size = (input_size // 2)
            input_size = (input_size - filter_size +3)
            input_size = (input_size - 2)//2 + 1
            
        self.fc = nn.Linear(input_channels * input_size * input_size, dense_units)
        self.out = nn.Linear(dense_units, num_classes)

    def forward(self, x):
        for layer in self.conv_layers:
            x = layer(x)

        x = torch.flatten(x, 1)
        x = torch.relu(self.fc(x))
        x = self.out(x)
        return x

In [6]:
sweep_config = {
    "method": "bayes",
    "name" : "sweep_cross_entropy",
    "project": "dl_assignment2",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        "num_filters": {"values": [32, 64, 128]},
        "filter_size": {"values": [3,5,7]},
        "dense_units": {"values": [128, 256, 512]},
        "activation_conv": {"values": ["relu", "gelu", "silu", "mish"]},
        "batch_size": {"values": [32,64]},
        "learning_rate": {"values": [0.001, 0.0001]},
        "lr_decay": {"values": [0.1, 0.5]},
        "num_epochs": {"value": 5},
        "data_augmentation": {"values": [True, False]},
        "batch_norm": {"values": [True, False]},
        "dropout": {"values": [0.2, 0.3]},
        "filter_organization": {"values": ["same", "double", "halve"]}
    }
}

In [7]:
def validate_model(model, val_loader, criterion, device):
    model.eval()  # Set model to evaluation mode
    val_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    # Calculate average loss and accuracy
    avg_val_loss = val_loss / len(val_loader.dataset)
    val_accuracy = 100 * correct / total
    
    return avg_val_loss, val_accuracy

In [8]:
def train_model():
    wandb.init()
    config = wandb.config

    # Load data
    train_loader, val_loader, num_classes = load_data(train_path, val_split=0.2, batch_size=config.batch_size, data_augmentation=config.data_augmentation)

    # Define model
    model = CNN(input_channels=3, num_classes=num_classes, num_filters=config.num_filters,
                filter_size=config.filter_size, dense_units=config.dense_units, activation_conv=config.activation_conv,
                data_augmentation=config.data_augmentation, batch_norm=config.batch_norm, dropout=config.dropout,
                filter_organization=config.filter_organization)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
    scheduler = StepLR(optimizer, step_size=1, gamma=config.lr_decay)

    # Train the model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    for epoch in range(config.num_epochs):
        model.train()
        train_loss = 0.0
        correct_train = 0
        total_train = 0
        
        # Training loop
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * images.size(0)
            
            # Calculate training accuracy
            _, predicted = torch.max(outputs, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()
        
        # Calculate training loss and accuracy
        avg_train_loss = train_loss / len(train_loader.dataset)
        train_accuracy = 100 * correct_train / total_train

        # Validation
        avg_val_loss, val_accuracy = validate_model(model, val_loader, criterion, device)

        # Log metrics to wandb
        run_name = "{}lr{}_batchsize{}_no_f{}_d_units{}_decay{}_data_aug{}_bnorm{}_dropout{}_fo{}".format(config.activation_conv, config.learning_rate, config.batch_size, config.num_filters, config.dense_units, config.lr_decay,
                            config.data_augmentation, config.batch_norm,config.dropout,config.filter_organization)
        wandb.run.name = run_name
        wandb.log({"epoch": epoch+1, 
                   "train_loss": avg_train_loss, 
                   "train_accuracy": train_accuracy, 
                   "val_loss": avg_val_loss, 
                   "val_accuracy": val_accuracy})
        
        print("Epoch: {}, Train Loss: {:.4f}, Train Acc: {:.2f}%, Val Loss: {:.4f}, Val Acc: {:.2f}%".format(
            epoch+1, avg_train_loss, train_accuracy, avg_val_loss, val_accuracy))
        
        # Adjust learning rate
        scheduler.step()


In [9]:
# Initialize and run the sweep
sweep_id = wandb.sweep(sweep_config, project="dl_assignment2")
wandb.agent(sweep_id, function=train_model,count=10)

Create sweep with ID: mhjdmph1
Sweep URL: https://wandb.ai/naveenpra/dl_assignment2/sweeps/mhjdmph1


wandb: Agent Starting Run: rth6qdlp with config:
wandb: 	activation_conv: silu
wandb: 	batch_norm: False
wandb: 	batch_size: 32
wandb: 	data_augmentation: True
wandb: 	dense_units: 256
wandb: 	dropout: 0.3
wandb: 	filter_organization: halve
wandb: 	filter_size: 3
wandb: 	learning_rate: 0.001
wandb: 	lr_decay: 0.5
wandb: 	num_epochs: 5
wandb: 	num_filters: 32
wandb: Currently logged in as: cs23m052 (naveenpra). Use `wandb login --relogin` to force relogin


Epoch: 1, Train Loss: 1.8374, Train Acc: 11.05%, Val Loss: 0.4533, Val Acc: 14.40%
Epoch: 2, Train Loss: 1.8081, Train Acc: 15.38%, Val Loss: 0.4427, Val Acc: 18.65%
Epoch: 3, Train Loss: 1.7910, Train Acc: 16.35%, Val Loss: 0.4407, Val Acc: 19.85%
Epoch: 4, Train Loss: 1.7924, Train Acc: 16.81%, Val Loss: 0.4398, Val Acc: 20.70%
Epoch: 5, Train Loss: 1.7861, Train Acc: 17.31%, Val Loss: 0.4398, Val Acc: 20.30%


epoch,▁▃▅▆█
train_accuracy,▁▆▇▇█
train_loss,█▄▂▂▁
val_accuracy,▁▆▇██
val_loss,█▂▁▁▁
epoch,5
train_accuracy,17.31466
train_loss,1.78614
val_accuracy,20.3
val_loss,0.43981


wandb: Agent Starting Run: 2gbh8w4h with config:
wandb: 	activation_conv: silu
wandb: 	batch_norm: False
wandb: 	batch_size: 32
wandb: 	data_augmentation: True
wandb: 	dense_units: 512
wandb: 	dropout: 0.3
wandb: 	filter_organization: double
wandb: 	filter_size: 5
wandb: 	learning_rate: 0.0001
wandb: 	lr_decay: 0.1
wandb: 	num_epochs: 5
wandb: 	num_filters: 32


...

Epoch: 1, Train Loss: 1.7244, Train Acc: 22.20%, Val Loss: 0.4182, Val Acc: 25.45%
Epoch: 2, Train Loss: 1.5975, Train Acc: 28.94%, Val Loss: 0.3954, Val Acc: 29.95%
Epoch: 3, Train Loss: 1.5478, Train Acc: 31.58%, Val Loss: 0.3880, Val Acc: 31.45%
Epoch: 4, Train Loss: 1.5157, Train Acc: 32.92%, Val Loss: 0.3810, Val Acc: 33.50%
Epoch: 5, Train Loss: 1.4944, Train Acc: 33.87%, Val Loss: 0.3782, Val Acc: 33.95%


epoch,▁▃▅▆█
train_accuracy,▁▅▇▇█
train_loss,█▄▃▂▁
val_accuracy,▁▅▆██
val_loss,█▄▃▁▁
epoch,5
train_accuracy,33.86673
train_loss,1.49441
val_accuracy,33.95
val_loss,0.3782


wandb: Agent Starting Run: xp08czyp with config:
wandb: 	activation_conv: mish
wandb: 	batch_norm: False
wandb: 	batch_size: 64
wandb: 	data_augmentation: True
wandb: 	dense_units: 128
wandb: 	dropout: 0.3
wandb: 	filter_organization: double
wandb: 	filter_size: 3
wandb: 	learning_rate: 0.0001
wandb: 	lr_decay: 0.5
wandb: 	num_epochs: 5
wandb: 	num_filters: 128


Epoch: 1, Train Loss: 1.7988, Train Acc: 15.49%, Val Loss: 0.4239, Val Acc: 23.25%
Epoch: 2, Train Loss: 1.6695, Train Acc: 25.70%, Val Loss: 0.4038, Val Acc: 30.20%
Epoch: 3, Train Loss: 1.6029, Train Acc: 29.32%, Val Loss: 0.3940, Val Acc: 32.00%
Epoch: 4, Train Loss: 1.5715, Train Acc: 31.79%, Val Loss: 0.3894, Val Acc: 33.50%
Epoch: 5, Train Loss: 1.5500, Train Acc: 32.72%, Val Loss: 0.3874, Val Acc: 33.55%


epoch,▁▃▅▆█
train_accuracy,▁▅▇██
train_loss,█▄▂▂▁
val_accuracy,▁▆▇██
val_loss,█▄▂▁▁
epoch,5
train_accuracy,32.71659
train_loss,1.54995
val_accuracy,33.55
val_loss,0.38736
